In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import h5py

In [5]:
cd /home/chenxupeng/projects/deepshape/

/home/chenxupeng/projects/deepshape


In [6]:
#test random convert to nan
with h5py.File('data/new/Spitale_2015_invivo_CDS_0.1') as f:
    y = f['y_test'][:]
y_f = y.flatten()
count = y_f.shape[0]
y_f = y.flatten()
count = y_f.shape[0]
count_convert = int(count*0.1)
index_notnan[0]
np.random.seed(1111)
np.random.shuffle(index_notnan[0])
pick_index = index_notnan[0][:count_convert]
y_f[pick_index] = np.nan
y_f.reshape(-1,128).shape

In [8]:
def append_extra_line(f):
    """Yield an empty line after the last line in the file
    """
    for line in f:
        yield line
    yield ''
def read_fasta(filename):
    with open(filename, 'r') as f:
        name = None
        seq = ''
        for line in append_extra_line(f):
            if line.startswith('>') or (len(line) == 0):
                if (len(seq) > 0) and (name is not None):
                    yield (name, seq)
                if line.startswith('>'):
                    name = line.strip()[1:].split()[0]
                    seq = ''
            else:
                if name is None:
                    raise ValueError('the first line does not start with ">"')
                seq += line.strip()

In [9]:
fasta = dict(read_fasta('data/fasta/CDS.transcript.fa'))

In [10]:
f =h5py.File('/home/chenxupeng/projects/deepshape/data/icSHAPE/Spitale_2015_invitro/CDS')

In [11]:
icshape = f['feature/icshape'][:]
name = f['name'][:]
start = f['start'][:]
end = f['end'][:]

In [12]:
icshape.shape

(12676378,)

In [ ]:
#percentile定5或者10

In [13]:
np.nanpercentile(icshape,95)

0.96899998188018799

In [16]:
def shape_tonan(percentile):
    low = np.nanpercentile(icshape,percentile)
    high = np.nanpercentile(icshape,100-percentile)
    index_nan = np.where(np.logical_and(icshape <high, icshape >low)==1)
    icshape_ = np.copy(icshape)
    icshape_[index_nan] = np.nan
    return icshape_

In [323]:
nan_icshape = shape_tonan(5)
coverage = 0.1

rna_icshape_whole = {}
rna_nan_index = {}
for i in tqdm(range(name.shape[0])):   #name.shape[0]
    rna_name = name[i]
    sequence = fasta[rna_name]
    shape_score = nan_icshape[start[i]:end[i]]
    length = end[i] - start[i] 
    count = length-128+1
    if count >=0:
        y = np.zeros([count,130]).astype('S')
        index = np.array([])
        for j in range(count):
            y[j][0] = rna_name
            y[j][1] = j
            y[j][2:] = shape_score[j:j+128]  
            if np.where(y[j][2:] == str(np.nan))[0].shape[0]/128.0 >1 - coverage:
                index = np.concatenate((index,np.array([j])))
        y = np.delete(y,index,axis =0)
        rna_icshape_whole[i] = y
        rna_nan_index[i] = index


  0%|          | 0/10123 [00:00<?, ?it/s]/home/chenxupeng/apps/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future

  1%|          | 60/10123 [00:02<05:59, 27.96it/s]


KeyboardInterrupt: 

In [321]:
#只需要上一部的key值那些i对应的rna 找fasta！ 一定要对应好！

{0: array([ 2736.,  2737.,  2738.,  2739.,  2740.,  2741.,  2742.,  2743.,
         2744.,  2745.,  2746.,  2747.,  2748.,  2749.,  2750.,  2751.,
         2752.,  2753.,  2754.,  2755.,  2756.,  2757.,  2758.,  2759.,
         2760.,  2761.,  2762.,  2763.,  2764.,  2765.,  2766.,  2767.,
         2768.,  2769.,  2770.,  2771.,  2772.,  2773.,  2774.,  2775.,
         2776.,  2777.,  2778.,  2779.,  2780.,  2781.,  2782.,  2783.,
         2784.,  2785.,  2786.,  2787.,  2788.,  2789.,  2790.,  2791.,
         2792.,  2793.,  2794.,  2795.,  2796.])}

In [17]:
data = fasta[fasta.keys()[0]]

In [18]:
#0 1 2 3 =》 one hot

In [19]:
data

'ATGACAGTCCAACCTTCTCCATGGTTTTCCGATCTCAGGCCCATGGCGACCTGCCCTGTCCTGCAGAAGGAGACACTGTTCCGCACAGGCGTCCATGCTTACAGAATCCCTGCTCTGCTCTACCTGAAGAAGCAGAAGACCCTGCTGGCCTTTGCGGAAAAGCGAGCCAGCAAGACGGATGAGCACGCAGAGTTGATTGTCCTGAGAAGAGGAAGCTACAACGAAGCCACCAACCGTGTCAAGTGGCAGCCTGAGGAAGTGGTGACCCAAGCCCAGCTGGAAGGCCACCGCTCCATGAATCCATGTCCCTTGTATGACAAGCAAACAAAGACCCTCTTCCTTTTCTTCATCGCTGTCCCTGGGCGTGTATCAGAACATCATCAGCTCCACACTAAGGTTAATGTCACACGGCTGTGCTGTGTCAGCAGCACTGACCATGGGAGGACCTGGAGCCCCATCCAGGACCTCACAGAGACCACCATTGGCAGCACTCATCAGGAATGGGCCACATTTGCTGTGGGTCCTGGGCATTGTCTGCAGCTGCGGAACCCAGCTGGGAGCCTGCTGGTACCTGCTTATGCCTACCGGAAACTGCACCCTGCTCAGAAGCCTACCCCCTTTGCCTTCTGCTTCATCAGCCTTGACCATGGGCACACATGGAAACTAGGCAACTTTGTGGCTGAAAACTCACTGGAGTGCCAGGTGGCTGAGGTTGGCACTGGAGCTCAGAGGATGGTATATCTCAATGCTAGGAGCTTCCTGGGAGCCAGGGTCCAGGCACAAAGTCCTAATGATGGTCTGGATTTCCAGGACAACCGGGTAGTGAGTAAGCTTGTAGAGCCCCCCCACGGGTGTCATGGAAGTGTGGTTGCCTTCCACAACCCCATCTCTAAGCCACATGCCTTAGACACATGGCTTCTTTATACACACCCTACAGACTCCAGGAATAGAACCAACCTGGGTGTGTACCTAAACCAGATGCCACTAGATCCCACAGCC

In [20]:
class Onehot(object):
    def __init__(self, alphabet='ATCG'):
        self.transtable = np.zeros(128, np.int64)
        self.transtable[[ord(a) for a in alphabet]] = range(len(alphabet))
        self.alphabet = np.frombuffer(alphabet, np.int8)
    def encode(self, seq):
        seq_int = self.transtable[np.frombuffer(seq, np.int8)]
        encoded = np.zeros((len(seq), len(self.alphabet)), np.int8)
        encoded[np.r_[:len(seq)], seq_int] = 1
        return encoded
    def decode(self, a):
        return self.alphabet[np.argmax(a, axis=1)].tostring()

In [26]:
onehot = Onehot()
encoded = onehot.encode(data)
#decoded = onehot.decode(encoded)

In [30]:
encoded[0:128]
#根据同名rna的索引i对应挑选片段  encoded[i:i+128]

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0,

In [168]:
#以cds的rna 10123条为顺序，一条一条rna来：
#每条rna切成128的小段，one hot，这是 128*4
#cds 文件怎么找icshape呢？  找到name的position i ,然后start[i] 是起始，end[i]是终止
#把这段icshape取出来， length， length -128 +1 为需要截取的次数 count
#for i in range(count): 取出y  把rna name 和在rna中的起始位置（i）加到前两维，算index: 用coverage给，如果非nan比例高于coverage的
#才予以保留   找到该条rna需要保留的i 
#然后构造X  X通过i找到起始位点，每次截取128的长度， 128*4  索引为rna name加上从第几个截取
#至此一个rna的处理完毕

<HDF5 dataset "start": shape (10123,), type "<i8">

In [ ]:
        self.sequences = dict(read_fasta(self.sequence_file))
        self.data = GenomicData(self.infile)
        if not self.feature:
            self.feature = self.data.features.keys()[0]
        values = self.data.features[self.feature]

        if (self.cutoff1 is None) and (self.cutoff2 is None):
            values_valid = values[np.logical_not(np.isnan(values))]
            self.cutoff1 = np.percentile(values_valid, self.percentile)
            self.cutoff2 = np.percentile(values_valid, 100 - self.percentile)
        if self.seed:
            self.logger.info('set random seed for numpy: {}'.format(self.seed))
            np.random.seed(self.seed)

        self.logger.info('discretize values with cutoffs: {}-{}'.format(self.cutoff1, self.cutoff2))

        not_nan_ind = np.nonzero(np.logical_not(np.isnan(values)))[0]
        one_ind = not_nan_ind[values[not_nan_ind] >= self.cutoff2]
        zero_ind = not_nan_ind[values[not_nan_ind] <= self.cutoff1]
        values[:] = np.nan
        values[one_ind] = 1
        values[zero_ind] = 0

        def save_dataset(filename, X_train, y_train, names_train,
                X_test, y_test, names_test):
            fout = h5py.File(filename, 'w')
            fout.create_dataset('names_train', data=names_train)
            fout.create_dataset('X_train', data=X_train)
            fout.create_dataset('y_train', data=y_train)
            fout.create_dataset('names_test', data=names_test)
            fout.create_dataset('X_test',  data=X_test)
            fout.create_dataset('y_test',  data=y_test)
            fout.create_dataset('offset', data=self.offset)
            fout.close()

        if 0 < self.train_test_split < 1:
            n_seqs = len(self.data.names)
            n_train = int(n_seqs*self.train_test_split)
            train_ind = np.full(n_seqs, False, dtype='bool')
            train_ind[np.random.choice(n_seqs, size=n_train, replace=False)] = True
            test_ind = np.logical_not(train_ind)
            names_train = self.data.names[train_ind]
            names_test = self.data.names[test_ind]

            if self.balance_kmer:
                Xs_train, ys_train, kmers = self.create_dataset(names_train, self.dense_output)
                Xs_test, ys_test, kmers = self.create_dataset(names_test, self.dense_output)
                for i in range(len(kmers)):
                    self.logger.info('number of training/test set for kmer {}: {}/{}'.format(
                        kmers[i], Xs_train[i].shape[0], Xs_test[i].shape[0]))
                if self.separate:
                    for i in range(len(kmers)):
                        nuc = self.alphabet[i]
                        outfile = os.path.join(self.outfile, kmers[i])
                        self.logger.info('save dataset for nucleotide {}: {}'.format(kmers[i], outfile))
                        prepare_output_file(outfile)
                        save_dataset(outfile, Xs_train[i], ys_train[i], names_train,
                            Xs_test[i], ys_test[i], names_test)
                else:
                    X_train = np.concatenate(Xs_train, axis=0)
                    y_train = np.concatenate(ys_train, axis=0)
                    ind = np.arange(X_train.shape[0])
                    np.random.shuffle(ind)
                    X_train = X_train[ind]
                    y_train = y_train[ind]

                    X_test = np.concatenate(Xs_test, axis=0)
                    y_test = np.concatenate(ys_test, axis=0)
                    ind = np.arange(X_test.shape[0])
                    np.random.shuffle(ind)
                    X_test = X_test[ind]
                    y_test = y_test[ind]
                    self.logger.info('save dataset with balanced nucletide composition: ' + self.outfile)
                    prepare_output_file(self.outfile)
                    save_dataset(self.outfile, X_train, y_train, names_train,
                        X_test, y_test, names_test)
            else:
                X_train, y_train = self.create_dataset(names_train, self.dense_output)
                X_test, y_test = self.create_dataset(names_test, self.dense_output)
                self.logger.info('save dataset: ' + self.outfile)
                prepare_output_file(self.outfile)
                save_dataset(self.outfile, X_train, y_train, names_train,
                    X_test, y_test, names_test)
        else:
            fout = h5py.File(self.outfile)
            X, y = self.create_dataset(self.data.names)
            fout.create_dataset('X', data=X)
            fout.create_dataset('y', data=y)
            fout.close()

In [24]:
f = h5py.File('/home/chenxupeng/projects/deepshape/data/icSHAPE/Spitale_2015_invitro/CDS')
end = f['end'][:]
icshape = f['feature/icshape'][:]
name = f['name'][:]
start= f['start'][:]
length = end -start

In [ ]:
#在生成y_test 的时候要标清楚每条的rna来源以及其起始为rna的第几个position

#剩下的都是有值的点，已经知道该片段的rna和起始位点了，下面用rna一条一条计算。y_pred和y_true 都对应到rna上
#最终的y_true不是CDS里读出来的标注，因为好多位置被转成了nan
#同步统计y_true 和y_pred  每条片段的rna对应回去，np.ndarray 一个rna长度的向量
#预计新的y_true 每行[rna,origin,128*ichshape]  name指rna name

#计算y_true
def rna_icshape_true():
    position_need = {}
    rna_icshape = {}
    #每条rna每个位置出现了几次
    rna_position = {}
    rna_ic_correct = {}
    for i in range(name.shape[0]):
        #每条rna
        rna_icshape[i] = np.zeros([length[i]])
        #每个片段看看是否匹配
        rna_position[i] = np.array([])
        for j in range(y_true.shape[0]):
            if y_true[j][0] = name[i]:
                origin = y_true[j][1]
                #找不是nan的部分,要累加
                index = np.where(np.isnan(y_true[2:])=0)[0]
                rna_icshape[i][origin:origin+128][index] += y_true[j][2:][index]
                #统计赋值的位置
                position = np.arange(origin:origin+128)[index]
                rna_position[i] = np.concatenate((rna_position[i],position))
        #统计第i条rna每个位置分别被加了几次
        position_count = np.zeros([length[i]])
        unique, counts = numpy.unique(rna_position[i], return_counts=True)
        position_count[unique] = counts
        #找到position count不是0的位置
        index_ = np.where(position_count)[0]
        rna_ic_correct[i] = rna_icshape[i][index_].astype('float') / position_count[index_]
        rna_ic_correct[i][np.where(rna_ic_correct>=0.5)] =1
        rna_ic_correct[i][np.where(rna_ic_correct<0.5)] =0
        position_need[i] = index_
    return rna_ic_correct,position_need

#计算y_test
def rna_icshape_pred():
    rna_icshape = {}
    #每条rna每个位置出现了几次
    rna_position = {}
    rna_ic_correct = {}
    for i in range(name.shape[0]):
        #每条rna
        rna_icshape[i] = np.zeros([length[i]])
        #每个片段看看是否匹配
        rna_position[i] = np.array([])
        for j in range(y_true.shape[0]):
            if y_true[j][0] = name[i]:
                origin = y_true[j][1]
                #找不是nan的部分,要累加
                index = np.where(np.isnan(y_true[2:])=0)[0]
                rna_icshape[i][origin:origin+128][index] += y_pred[j][index]
                #统计赋值的位置
                position = np.arange(origin:origin+128)[index]
                rna_position[i] = np.concatenate((rna_position[i],position))
        #统计第i条rna每个位置分别被加了几次
        position_count = np.zeros([length[i]])
        unique, counts = numpy.unique(rna_position[i], return_counts=True)
        position_count[unique] = counts
        #找到position count不是0的位置
        index_ = np.where(position_count)[0]
        rna_ic_correct[i] = rna_icshape[i][index_].astype('float') / position_count[index_]
        rna_ic_correct[i][np.where(rna_ic_correct>=0.5)] =1
        rna_ic_correct[i][np.where(rna_ic_correct<0.5)] =0
    return rna_ic_correct

In [ ]:
#看下 返回的样子  需要返回一个真正被考虑的位置的index，要不然nan和0都是0，acc肯定大！ 
#读取 true和pred  以及每条rna需要被考虑的位点

In [ ]:
#分类计算acc  对应出来染色体名字和所属类别
category = []
acc = {}
for j in range(len(category)):
    acc[category[j]] = {}
    for i in range(name.shape[0]):
        if name[i] = category[j]:
            count = position_need[i].shape[0]    
            acc[category[j]][name[i]] = (count - np.sum(rna_ic_correct_true - rna_ic_correct_pred))/count    